In [32]:
using Pkg; Pkg.activate(".")

  Activating project at `D:\Documents\School Projects\MTH8408\Projet`


In [33]:
Pkg.instantiate()

In [34]:
Pkg.add("CSV"); Pkg.add("DataFrames"); Pkg.add("LsqFit"); Pkg.add("Plots"); Pkg.add("Krylov");

   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`
   Resolving package versions...
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Project.toml`
  No Changes to `D:\Documents\School Projects\MTH8408\Projet\Manifest.toml`


In [35]:
using DataFrames, LsqFit, CSV, Statistics, QuadraticModels, LinearAlgebra, SparseMatricesCOO, Krylov, Dates

## Exploration des données

In [36]:
data_folder = "Data/"
file_name   = "2947732.csv" #"weatherstats_montreal_daily.csv" ou "2947732.csv"
ville       = "Dakar" #"Montréal" ou "Dakar"
df = DataFrame(CSV.File(data_folder * file_name))
first(df, 5)

,STATION,NAME,DATE,TAVG
,String15,String15,Date,Int64
1,SG000061641,"DAKAR YOFF, SG",1973-01-02,67
2,SG000061641,"DAKAR YOFF, SG",1973-01-03,69
3,SG000061641,"DAKAR YOFF, SG",1973-01-04,68
4,SG000061641,"DAKAR YOFF, SG",1973-01-06,70
5,SG000061641,"DAKAR YOFF, SG",1973-01-07,71


In [37]:
to_C(x) = (x-32) *5 /9

to_C (generic function with 1 method)

In [38]:
if(ville == "Dakar")
    rename!(df,:DATE => :date)
    rename!(df,:TAVG => :avg_temperature)
    df.avg_temperature = to_C.(df.avg_temperature)
elseif (ville == "Montréal")
    df = reverse(df)
end
last(df.date)

2022-04-15

In [39]:
nrow(df)

17816

In [40]:
ncol(df)

4

#### Train Test split

In [41]:
date1, date2 = Date("1980-01-01"), Date("2015-01-01")
train_df = df[date2 .> df.date .>= date1, :]
test_df  = df[df.date .>= Date("2015-01-01"), :]
last(test_df)

,STATION,NAME,date,avg_temperature
,String15,String15,Date,Float64
2657,SG000061641,"DAKAR YOFF, SG",2022-04-15,21.6667


In [42]:
first(train_df, 5)

,STATION,NAME,date,avg_temperature
,String15,String15,Date,Float64
1,SG000061641,"DAKAR YOFF, SG",1980-01-01,23.3333
2,SG000061641,"DAKAR YOFF, SG",1980-01-02,22.7778
3,SG000061641,"DAKAR YOFF, SG",1980-01-03,22.2222
4,SG000061641,"DAKAR YOFF, SG",1980-01-04,22.2222
5,SG000061641,"DAKAR YOFF, SG",1980-01-05,21.6667


In [43]:
first(test_df, 5)

,STATION,NAME,date,avg_temperature
,String15,String15,Date,Float64
1,SG000061641,"DAKAR YOFF, SG",2015-01-01,22.2222
2,SG000061641,"DAKAR YOFF, SG",2015-01-02,22.2222
3,SG000061641,"DAKAR YOFF, SG",2015-01-03,23.3333
4,SG000061641,"DAKAR YOFF, SG",2015-01-04,23.8889
5,SG000061641,"DAKAR YOFF, SG",2015-01-05,22.7778


In [44]:
## Get the data
jours_train = [i for i in 1:length(train_df.avg_temperature)]
temp_train  = train_df.avg_temperature
jours_test  = [last(jours_train) + i for i in 1:length(test_df.avg_temperature)]
temp_test   = test_df.avg_temperature

2657-element Vector{Float64}:
 22.22222222222222
 22.22222222222222
 23.333333333333332
 23.88888888888889
 22.77777777777778
 22.22222222222222
 21.666666666666668
 21.11111111111111
 21.11111111111111
 21.11111111111111
 21.666666666666668
 21.11111111111111
 20.555555555555557
  ⋮
 20.555555555555557
 21.11111111111111
 21.666666666666668
 22.77777777777778
 23.88888888888889
 22.77777777777778
 21.666666666666668
 20.555555555555557
 20.555555555555557
 21.11111111111111
 21.666666666666668
 21.666666666666668

In [45]:
last(jours_train)

12766

In [46]:
jours_test[1]

12767

### Création des matrices associées aux modèles

$$f_s(x, d) = x_1 + x_2d + x_3 cos(\cfrac{2 \pi d}{365.25})+ x_4 sin(\cfrac{2\pi d}{365.25}) + x_5 cos(\cfrac{2\pi d}{10.7 \times 365.25})+ x_6 sin(\cfrac{2\pi d}{10.7 \times 365.25}); \forall (x, d) \in \mathbb{R}^6 \times \mathbb{R}.$$

$$f_h(x, d) = x_1 + x_2d + x_3 cos(\cfrac{2 \pi d}{365.25})+ x_4 sin(\cfrac{2\pi d}{365.25}) + x_5 cos(\cfrac{2\pi d}{10.7 \times 365.25})+ x_6 sin(\cfrac{2\pi d}{10.7 \times 365.25}) + x_7 cos(\cfrac{4 \pi d}{365.25})+ x_8 sin(\cfrac{4\pi d}{365.25}); \forall (x, d) \in \mathbb{R}^8 \times \mathbb{R}.$$

Chaque colonne de la matrice $A_s$ du modèle simple $f_s$ représente un membre de la fonction polynomiale en $x$. On minimise donc :
$$\underset{x \in \mathbb{R}^6 }{min}||A_s x - b||_2.$$

Pareillement pour $A_h$ et le modèle avec les harmoniques $f_h$.

In [47]:
A_S = [ones(length(jours_train)) jours_train cos.(2*pi*jours_train/365.25) sin.(2*pi*jours_train/365.25) cos.(2*pi*jours_train/(10.7*365.25)) sin.(2*pi*jours_train/(10.7*365.25))]
A_H = [ones(length(jours_train)) jours_train cos.(2*pi*jours_train/365.25) sin.(2*pi*jours_train/365.25) cos.(2*pi*jours_train/(10.7*365.25)) sin.(2*pi*jours_train/(10.7*365.25)) cos.(4*pi*jours_train/(10.7*365.25)) sin.(4*pi*jours_train/(10.7*365.25))]

A_S_test = [ones(length(jours_test)) jours_test cos.(2*pi*jours_test/365.25) sin.(2*pi*jours_test/365.25) cos.(2*pi*jours_test/(10.7*365.25)) sin.(2*pi*jours_test/(10.7*365.25))]
A_H_test = [ones(length(jours_test)) jours_test cos.(2*pi*jours_test/365.25) sin.(2*pi*jours_test/365.25) cos.(2*pi*jours_test/(10.7*365.25)) sin.(2*pi*jours_test/(10.7*365.25)) cos.(4*pi*jours_test/(10.7*365.25)) sin.(4*pi*jours_test/(10.7*365.25))]

2657×8 Matrix{Float64}:
 1.0  12767.0  0.958774  -0.28417    …   0.994472  -0.97795   -0.208839
 1.0  12768.0  0.96352   -0.267636       0.994302  -0.977273  -0.211983
 1.0  12769.0  0.967981  -0.251022       0.994129  -0.976587  -0.215124
 1.0  12770.0  0.972156  -0.234334       0.993954  -0.97589   -0.218263
 1.0  12771.0  0.976043  -0.217577       0.993776  -0.975183  -0.2214
 1.0  12772.0  0.979641  -0.200755   …   0.993596  -0.974466  -0.224534
 1.0  12773.0  0.98295   -0.183874       0.993413  -0.973739  -0.227666
 1.0  12774.0  0.985967  -0.166938       0.993228  -0.973002  -0.230796
 1.0  12775.0  0.988693  -0.149953       0.99304   -0.972255  -0.233924
 1.0  12776.0  0.991126  -0.132924       0.992849  -0.971498  -0.237049
 1.0  12777.0  0.993266  -0.115856   …   0.992656  -0.970731  -0.240171
 1.0  12778.0  0.995112  -0.0987527      0.99246   -0.969953  -0.243291
 1.0  12779.0  0.996663  -0.0816206      0.992262  -0.969166  -0.246409
 ⋮                                   ⋱   ⋮

#### Solving Simple Model

In [48]:
(x_s, stats) = Krylov.cgls(A_S, temp_train)

([23.8113539114354, 8.586517218558154e-5, -2.2807755309041413, -3.0239449953105195, -0.025811975903185586, -0.0022508610843897736], Simple stats
 niter: 7
 solved: true
 inconsistent: false
 residuals: []
 Aresiduals: []
 κ₂(A): []
 status: solution good enough given atol and rtol
)

##### RSS $||Ax -b||_2^2$

In [49]:
RSS_S = norm(A_S*x_s - temp_train, 2)^2

23813.142810470457

In [50]:
RSS_S_test = norm(A_S_test*x_s - temp_test, 2)^2

5607.653724192067

In [51]:
print(RSS_S/length(temp_train), " et pour l'ensemble test ", RSS_S_test/length(temp_test))

1.8653566356314004 et pour l'ensemble test 2.1105207844155314

#### Solving Harmonic Model

In [52]:
(x_h, stats) = Krylov.cgls(A_H, temp_train)

([23.812286377951963, 8.555933613713956e-5, -2.280174709455148, -3.02279253573349, -0.02491388605867672, -0.0060306472450724815, -0.0630992753666203, 0.018361366048328526], Simple stats
 niter: 7
 solved: true
 inconsistent: false
 residuals: []
 Aresiduals: []
 κ₂(A): []
 status: solution good enough given atol and rtol
)

##### RSS $||Ax -b||_2^2$

In [53]:
RSS_H = norm(A_H*x_h - temp_train,2 )^2

23785.92004358601

In [54]:
RSS_H_test = norm(A_H_test*x_h - temp_test,2 )^2

5607.211142108986

In [55]:
print(RSS_H/length(temp_train), " et pour l'ensemble test ", RSS_H_test/length(temp_test))

1.8632241926669286 et pour l'ensemble test 2.110354212310495

#### Comparaison du modèle Simple et du modèle Harmonique

In [56]:
RSS_H < RSS_S

true

In [57]:
RSS_H_test < RSS_S_test

true

Le modèle harmonique est plus performant sur les données **test** et **train**.

### Plotting Results

In [58]:
pred_S = vcat(A_S*x_s, A_S_test*x_s)
pred_H = vcat(A_H*x_h, A_H_test*x_h)
dates  = vcat(train_df.date, test_df.date)
temp   = vcat(temp_train, temp_test)

15423-element Vector{Float64}:
 23.333333333333332
 22.77777777777778
 22.22222222222222
 22.22222222222222
 21.666666666666668
 21.666666666666668
 21.11111111111111
 21.11111111111111
 20.0
 20.555555555555557
 21.11111111111111
 22.22222222222222
 21.666666666666668
  ⋮
 20.555555555555557
 21.11111111111111
 21.666666666666668
 22.77777777777778
 23.88888888888889
 22.77777777777778
 21.666666666666668
 20.555555555555557
 20.555555555555557
 21.11111111111111
 21.666666666666668
 21.666666666666668

In [59]:
using Plots
gr()
scatter(dates, temp, label="Température de " * ville, markersize=1, markercolor="blue")
p = plot!(dates, [pred_S pred_H], title="Prédiction des Modèles pour "*ville, label=["Modèle Simple" "Modèle avec Harmoniques"], linecolor=["red" "cyan"], legend=:bottomright)
xlabel!("Time t")
ylabel!("Temperature C")
plot!(size=(1000,400))
savefig("Results/Two_Models_"* ville *".png")